In [2]:
!pip -q install kaggle torch torchvision pandas scikit-learn tqdm matplotlib


In [3]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aabhasgurung","key":"8ea4e162f39b2f0c9b34750da5c3e6f0"}'}

# New section

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -la ~/.kaggle


total 16
drwxr-xr-x 2 root root 4096 Jan  5 09:46 .
drwx------ 1 root root 4096 Jan  5 09:46 ..
-rw------- 1 root root   68 Jan  5 09:46 kaggle.json


In [5]:
!kaggle datasets download -d ziya07/multimodal-e-commerce-dataset -p /content --force
!unzip -q /content/multimodal-e-commerce-dataset.zip -d /content/dataset
!ls -lah /content/dataset | head -n 50


Dataset URL: https://www.kaggle.com/datasets/ziya07/multimodal-e-commerce-dataset
License(s): CC0-1.0
 99% 2.39G/2.41G [00:22<00:00, 177MB/s]
100% 2.41G/2.41G [00:22<00:00, 116MB/s]
total 61M
drwxr-xr-x 3 root root 4.0K Jan  5 09:47 .
drwxr-xr-x 1 root root 4.0K Jan  5 09:47 ..
drwxr-xr-x 3 root root 4.0K Jan  5 09:47 images
-rw-r--r-- 1 root root 8.5M Aug  9 10:10 X_test_update.csv
-rw-r--r-- 1 root root  52M Aug  9 10:10 X_train_update.csv
-rw-r--r-- 1 root root 886K Aug  9 10:10 Y_train_CVw08PX.csv


In [6]:
!ls -lah /content


total 2.5G
drwxr-xr-x 1 root root 4.0K Jan  5 09:47 .
drwxr-xr-x 1 root root 4.0K Jan  5 09:38 ..
drwxr-xr-x 4 root root 4.0K Dec  9 14:41 .config
drwxr-xr-x 3 root root 4.0K Jan  5 09:47 dataset
-rw-r--r-- 1 root root   68 Jan  5 09:46 kaggle.json
-rw-r--r-- 1 root root 2.5G Aug  9 10:13 multimodal-e-commerce-dataset.zip
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data


In [7]:
!ls -lah /content/dataset | head -n 50


total 61M
drwxr-xr-x 3 root root 4.0K Jan  5 09:47 .
drwxr-xr-x 1 root root 4.0K Jan  5 09:47 ..
drwxr-xr-x 3 root root 4.0K Jan  5 09:47 images
-rw-r--r-- 1 root root 8.5M Aug  9 10:10 X_test_update.csv
-rw-r--r-- 1 root root  52M Aug  9 10:10 X_train_update.csv
-rw-r--r-- 1 root root 886K Aug  9 10:10 Y_train_CVw08PX.csv


In [8]:
import pandas as pd

x_train_path = "/content/dataset/X_train_update.csv"
y_train_path = "/content/dataset/Y_train_CVw08PX.csv"

x_train = pd.read_csv(x_train_path)
y_train = pd.read_csv(y_train_path)

x_train = x_train.rename(columns={"Unnamed: 0": "idx"})
y_train = y_train.rename(columns={"Unnamed: 0": "idx"})

print("X_train columns:", list(x_train.columns))
print("Y_train columns:", list(y_train.columns))

train_df = x_train.merge(y_train, on="idx", how="inner")

print("Merged train_df shape:", train_df.shape)
train_df.head(3)



X_train columns: ['idx', 'designation', 'description', 'productid', 'imageid']
Y_train columns: ['idx', 'prdtypecode']
Merged train_df shape: (84916, 6)


,idx,designation,description,productid,imageid,prdtypecode
0,0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10
1,1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280
2,2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50


In [9]:
y_train.head(3)


,idx,prdtypecode
0,0,10
1,1,2280
2,2,50


In [10]:

shared_cols = sorted(set(x_train.columns).intersection(set(y_train.columns)))
shared_cols


['idx']

In [11]:

candidates = ["id", "ID", "product_id", "image_id", "idx"]
join_key = None
for c in candidates:
    if c in x_train.columns and c in y_train.columns:
        join_key = c
        break

if join_key is None:
    print("Could not auto-detect join key.")
    print("Shared columns:", sorted(set(x_train.columns).intersection(set(y_train.columns))))
    raise ValueError("Tell me the shared column you want to join on.")
else:
    print("Using join key:", join_key)

train_df = x_train.merge(y_train, on=join_key, how="inner")
print("Merged train_df shape:", train_df.shape)
train_df.head(3)


Using join key: idx
Merged train_df shape: (84916, 6)


,idx,designation,description,productid,imageid,prdtypecode
0,0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10
1,1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280
2,2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50


In [12]:
import pandas as pd


train_df["designation"] = train_df["designation"].fillna("")
train_df["description"] = train_df["description"].fillna("")

train_df["text"] = (train_df["designation"].astype(str) + " " + train_df["description"].astype(str)).str.strip()

label_col = "prdtypecode"

train_df[["text", label_col, "imageid"]].head(5)


,text,prdtypecode,imageid
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,10,1263597046
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,2280,1008141237
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,50,938777978
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,1280,457047496
4,La Guerre Des Tuques Luc a des id&eacute;es de...,2705,1077757786


In [13]:
import os
import glob

IMAGES_DIR = "/content/dataset/images"

all_imgs = glob.glob(os.path.join(IMAGES_DIR, "**", "*.*"), recursive=True)

print("Total image files found:", len(all_imgs))
print("First 20 files:")
for p in all_imgs[:20]:
    print(p)


Total image files found: 98728
First 20 files:
/content/dataset/images/images/image_test/image_1007556805_product_435090339.jpg
/content/dataset/images/images/image_test/image_1169904074_product_2887191335.jpg
/content/dataset/images/images/image_test/image_1267161947_product_3941810464.jpg
/content/dataset/images/images/image_test/image_1167118084_product_364387361.jpg
/content/dataset/images/images/image_test/image_1276225055_product_4012179080.jpg
/content/dataset/images/images/image_test/image_1273780308_product_3992809817.jpg
/content/dataset/images/images/image_test/image_1255204788_product_3863215844.jpg
/content/dataset/images/images/image_test/image_1121397674_product_2032708642.jpg
/content/dataset/images/images/image_test/image_652832876_product_53217499.jpg
/content/dataset/images/images/image_test/image_1257269386_product_3879903943.jpg
/content/dataset/images/images/image_test/image_1324015913_product_4235887963.jpg
/content/dataset/images/images/image_test/image_13100056

In [14]:
import glob, os
import re

IMAGES_DIR = "/content/dataset/images"


all_imgs = glob.glob(os.path.join(IMAGES_DIR, "**", "*.*"), recursive=True)
print("Total image files found:", len(all_imgs))


id_to_path = {}

for p in all_imgs:
    fname = os.path.basename(p)
    base = os.path.splitext(fname)[0]

    id_to_path[base] = p

    match = re.search(r'image_(\d+)_product_(\d+)', base)
    if match:
        image_id_from_filename = match.group(1)
        id_to_path.setdefault(image_id_from_filename, p)

print("Built id_to_path entries:", len(id_to_path))

train_df["image_path"] = train_df["imageid"].astype(str).map(id_to_path)

missing = train_df["image_path"].isna().sum()
print("Missing images after mapping:", missing, "/", len(train_df))

train_df[["imageid", "image_path"]].head(5)

Total image files found: 98728
Built id_to_path entries: 197456
Missing images after mapping: 0 / 84916


,imageid,image_path
0,1263597046,/content/dataset/images/images/image_train/ima...
1,1008141237,/content/dataset/images/images/image_train/ima...
2,938777978,/content/dataset/images/images/image_train/ima...
3,457047496,/content/dataset/images/images/image_train/ima...
4,1077757786,/content/dataset/images/images/image_train/ima...


In [15]:
import os, glob

IMAGES_DIR = "/content/dataset/images"
all_imgs = glob.glob(os.path.join(IMAGES_DIR, "**", "*.*"), recursive=True)

print("Example image files:")
for p in all_imgs[:30]:
    print(os.path.relpath(p, IMAGES_DIR))


Example image files:
images/image_test/image_1007556805_product_435090339.jpg
images/image_test/image_1169904074_product_2887191335.jpg
images/image_test/image_1267161947_product_3941810464.jpg
images/image_test/image_1167118084_product_364387361.jpg
images/image_test/image_1276225055_product_4012179080.jpg
images/image_test/image_1273780308_product_3992809817.jpg
images/image_test/image_1255204788_product_3863215844.jpg
images/image_test/image_1121397674_product_2032708642.jpg
images/image_test/image_652832876_product_53217499.jpg
images/image_test/image_1257269386_product_3879903943.jpg
images/image_test/image_1324015913_product_4235887963.jpg
images/image_test/image_1310005610_product_4181948446.jpg
images/image_test/image_973392327_product_268799190.jpg
images/image_test/image_1274721096_product_89618619.jpg
images/image_test/image_977934906_product_278992868.jpg
images/image_test/image_1022019601_product_528539862.jpg
images/image_test/image_1138374917_product_2366596631.jpg
image

In [16]:
import glob, os

IMAGES_DIR = "/content/dataset/images"
sample_id = str(train_df["imageid"].iloc[0])

hits = glob.glob(os.path.join(IMAGES_DIR, "**", f"*{sample_id}*"), recursive=True)
print("Sample imageid:", sample_id)
print("Hits found:", len(hits))
if hits[:10]:
    for h in hits[:10]:
        print(os.path.relpath(h, IMAGES_DIR))


Sample imageid: 1263597046
Hits found: 1
images/image_train/image_1263597046_product_3804725264.jpg


In [17]:
import os, glob

IMAGES_DIR = "/content/dataset/images"

for idx_val in train_df["idx"].head(5).tolist():
    idx_val = str(idx_val)

    candidates = [
        os.path.join(IMAGES_DIR, f"{idx_val}.jpg"),
        os.path.join(IMAGES_DIR, f"{idx_val}.png"),
        os.path.join(IMAGES_DIR, f"image_{idx_val}.jpg"),
        os.path.join(IMAGES_DIR, f"image_{idx_val}.png"),
    ]

    found = [p for p in candidates if os.path.exists(p)]
    print("idx:", idx_val, "->", "FOUND" if found else "not found", found[:1])


idx: 0 -> not found []
idx: 1 -> not found []
idx: 2 -> not found []
idx: 3 -> not found []
idx: 4 -> not found []


In [18]:
import os, glob, re
import pandas as pd

IMAGES_DIR = "/content/dataset/images"

all_imgs = glob.glob(os.path.join(IMAGES_DIR, "**", "*.*"), recursive=True)
print("Total image files:", len(all_imgs))
print("Sample filenames:")
for p in all_imgs[:20]:
    print(os.path.relpath(p, IMAGES_DIR))

token_to_path = {}
for p in all_imgs:
    base = os.path.splitext(os.path.basename(p))[0]
    tokens = re.findall(r"\d+", base)
    for tok in tokens:

        token_to_path.setdefault(tok.lstrip("0") or "0", p)

print("token_to_path size:", len(token_to_path))

candidates = ["imageid", "idx", "productid"]
for col in candidates:
    if col not in train_df.columns:
        continue
    mapped = train_df[col].astype(str).map(lambda x: token_to_path.get(x.lstrip("0") or "0"))
    missing = mapped.isna().sum()
    print(f"Mapping with {col}: missing {missing} / {len(train_df)}")

best_col = None
best_missing = 10**18
for col in candidates:
    if col not in train_df.columns:
        continue
    mapped = train_df[col].astype(str).map(lambda x: token_to_path.get(x.lstrip("0") or "0"))
    missing = mapped.isna().sum()
    if missing < best_missing:
        best_missing = missing
        best_col = col

print("\Best column for image mapping:", best_col, "with missing:", best_missing)

train_df["image_path"] = train_df[best_col].astype(str).map(lambda x: token_to_path.get(x.lstrip("0") or "0"))

print("Final missing image_path:", train_df["image_path"].isna().sum(), "/", len(train_df))
train_df[["idx", "imageid", "productid", "image_path"]].head(10)


<>:41: SyntaxWarning: invalid escape sequence '\B'
<>:41: SyntaxWarning: invalid escape sequence '\B'
/tmp/ipython-input-746432221.py:41: SyntaxWarning: invalid escape sequence '\B'
  print("\Best column for image mapping:", best_col, "with missing:", best_missing)


Total image files: 98728
Sample filenames:
images/image_test/image_1007556805_product_435090339.jpg
images/image_test/image_1169904074_product_2887191335.jpg
images/image_test/image_1267161947_product_3941810464.jpg
images/image_test/image_1167118084_product_364387361.jpg
images/image_test/image_1276225055_product_4012179080.jpg
images/image_test/image_1273780308_product_3992809817.jpg
images/image_test/image_1255204788_product_3863215844.jpg
images/image_test/image_1121397674_product_2032708642.jpg
images/image_test/image_652832876_product_53217499.jpg
images/image_test/image_1257269386_product_3879903943.jpg
images/image_test/image_1324015913_product_4235887963.jpg
images/image_test/image_1310005610_product_4181948446.jpg
images/image_test/image_973392327_product_268799190.jpg
images/image_test/image_1274721096_product_89618619.jpg
images/image_test/image_977934906_product_278992868.jpg
images/image_test/image_1022019601_product_528539862.jpg
images/image_test/image_1138374917_produc

,idx,imageid,productid,image_path
0,0,1263597046,3804725264,/content/dataset/images/images/image_train/ima...
1,1,1008141237,436067568,/content/dataset/images/images/image_train/ima...
2,2,938777978,201115110,/content/dataset/images/images/image_train/ima...
3,3,457047496,50418756,/content/dataset/images/images/image_train/ima...
4,4,1077757786,278535884,/content/dataset/images/images/image_train/ima...
5,5,393356830,5862738,/content/dataset/images/images/image_train/ima...
6,6,907794536,91920807,/content/dataset/images/images/image_train/ima...
7,7,999581347,344240059,/content/dataset/images/images/image_train/ima...
8,8,1325918866,4239126071,/content/dataset/images/images/image_train/ima...
9,9,1245644185,3793572222,/content/dataset/images/images/image_train/ima...


In [19]:
from sklearn.model_selection import train_test_split
import pandas as pd

label_col = "prdtypecode"

df = train_df[["image_path", "text", label_col]].copy()

labels = sorted(df[label_col].unique().tolist())
label2id = {lbl: i for i, lbl in enumerate(labels)}
id2label = {i: lbl for lbl, i in label2id.items()}

df["label_id"] = df[label_col].map(label2id)

print("Num classes:", len(label2id))
print(df[["image_path","text",label_col,"label_id"]].head(3))


Num classes: 27
                                          image_path  \
0  /content/dataset/images/images/image_train/ima...   
1  /content/dataset/images/images/image_train/ima...   
2  /content/dataset/images/images/image_train/ima...   

                                                text  prdtypecode  label_id  
0  Olivia: Personalisiertes Notizbuch / 150 Seite...           10         0  
1  Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...         2280        18  
2  Grand Stylet Ergonomique Bleu Gamepad Nintendo...           50         2  


In [20]:
train_df2, val_df2 = train_test_split(
    df,
    test_size=0.15,
    random_state=42,
    stratify=df["label_id"]
)

print("Train:", train_df2.shape, "Val:", val_df2.shape)

train_df2.to_csv("/content/train_split.csv", index=False)
val_df2.to_csv("/content/val_split.csv", index=False)

print("Saved: /content/train_split.csv and /content/val_split.csv")


Train: (72178, 4) Val: (12738, 4)
Saved: /content/train_split.csv and /content/val_split.csv


In [21]:
!pip -q install transformers torchvision


In [22]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from transformers import AutoTokenizer

MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

val_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

class EcomDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=64, tf=None):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.tf = tf

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Image
        img = Image.open(row["image_path"]).convert("RGB")
        if self.tf:
            img = self.tf(img)

        # Text
        text = str(row["text"])
        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt",
        )

        item = {
            "image": img,
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "label": torch.tensor(int(row["label_id"]), dtype=torch.long),
        }
        return item

train_ds = EcomDataset(train_df2, tokenizer, max_len=64, tf=train_tf)
val_ds   = EcomDataset(val_df2, tokenizer, max_len=64, tf=val_tf)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

print("Dataset ready")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Dataset ready


In [23]:
batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, v.shape, v.dtype)


image torch.Size([16, 3, 224, 224]) torch.float32
input_ids torch.Size([16, 64]) torch.int64
attention_mask torch.Size([16, 64]) torch.int64
label torch.Size([16]) torch.int64


In [24]:
import numpy as np

FAST_TRAIN = 10000
FAST_VAL = 2000

train_small = train_df2.sample(n=FAST_TRAIN, random_state=42)
val_small   = val_df2.sample(n=FAST_VAL, random_state=42)

train_ds = EcomDataset(train_small, tokenizer, max_len=64, tf=train_tf)
val_ds   = EcomDataset(val_small, tokenizer, max_len=64, tf=val_tf)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

print(len(train_ds), len(val_ds))


10000 2000


In [25]:
import torch
import torch.nn as nn
from torchvision import models
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

NUM_CLASSES = len(label2id)

class ImageOnlyModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        m = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        in_features = m.fc.in_features
        m.fc = nn.Identity()  # remove classifier
        self.backbone = m
        self.head = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )

    def forward(self, image, **kwargs):
        feats = self.backbone(image)
        return self.head(feats)

def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)
    return {"accuracy": acc, "precision_macro": p, "recall_macro": r, "f1_macro": f1}

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    all_true, all_pred = [], []

    for batch in tqdm(loader, desc="train"):
        image = batch["image"].to(device)
        label = batch["label"].to(device)

        optimizer.zero_grad()
        logits = model(image=image)
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1).detach().cpu().tolist()
        all_pred.extend(preds)
        all_true.extend(label.detach().cpu().tolist())

    return total_loss / len(loader), compute_metrics(all_true, all_pred)

@torch.no_grad()
def eval_one_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_true, all_pred = [], []

    for batch in tqdm(loader, desc="val"):
        image = batch["image"].to(device)
        label = batch["label"].to(device)

        logits = model(image=image)
        loss = criterion(logits, label)

        total_loss += loss.item()
        preds = logits.argmax(dim=1).detach().cpu().tolist()
        all_pred.extend(preds)
        all_true.extend(label.detach().cpu().tolist())

    return total_loss / len(loader), compute_metrics(all_true, all_pred)

model = ImageOnlyModel(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)

train_loss, train_metrics = train_one_epoch(model, train_loader, optimizer, criterion)
val_loss, val_metrics     = eval_one_epoch(model, val_loader, criterion)

print("\nImage-only baseline results")
print("Train loss:", train_loss, train_metrics)
print("Val loss:", val_loss, val_metrics)


Device: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 119MB/s]
val: 100%|██████████| 125/125 [00:17<00:00,  7.01it/s]



Image-only baseline results
Train loss: 2.145473635482788 {'accuracy': 0.387, 'precision_macro': 0.2891850073853053, 'recall_macro': 0.2589333406063956, 'f1_macro': 0.2491865413660196}
Val loss: 1.8904210019111634 {'accuracy': 0.4605, 'precision_macro': 0.3511194699081855, 'recall_macro': 0.357151364839336, 'f1_macro': 0.33264968175914117}


In [26]:
import torch
import torch.nn as nn
from transformers import AutoModel
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

NUM_CLASSES = len(label2id)

class TextOnlyModel(nn.Module):
    def __init__(self, num_classes, model_name="distilbert-base-uncased", dropout=0.2):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden = self.encoder.config.hidden_size
        self.head = nn.Sequential(
            nn.Linear(hidden, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask, **kwargs):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls = out.last_hidden_state[:, 0, :]  # CLS token
        return self.head(cls)

def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="macro", zero_division=0
    )
    return {"accuracy": acc, "precision_macro": p, "recall_macro": r, "f1_macro": f1}

def train_one_epoch_text(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    all_true, all_pred = [], []

    for batch in tqdm(loader, desc="train (text-only)"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label = batch["label"].to(device)

        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1).detach().cpu().tolist()
        all_pred.extend(preds)
        all_true.extend(label.detach().cpu().tolist())

    return total_loss / len(loader), compute_metrics(all_true, all_pred)

@torch.no_grad()
def eval_one_epoch_text(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_true, all_pred = [], []

    for batch in tqdm(loader, desc="val (text-only)"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label = batch["label"].to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(logits, label)

        total_loss += loss.item()
        preds = logits.argmax(dim=1).detach().cpu().tolist()
        all_pred.extend(preds)
        all_true.extend(label.detach().cpu().tolist())

    return total_loss / len(loader), compute_metrics(all_true, all_pred)

text_model = TextOnlyModel(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(text_model.parameters(), lr=2e-5, weight_decay=1e-2)

train_loss, train_metrics = train_one_epoch_text(text_model, train_loader, optimizer, criterion)
val_loss, val_metrics     = eval_one_epoch_text(text_model, val_loader, criterion)

print("\nText-only baseline results")
print("Train loss:", train_loss, train_metrics)
print("Val loss:", val_loss, val_metrics)


Device: cuda


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

val (text-only): 100%|██████████| 125/125 [00:13<00:00,  9.03it/s]


Text-only baseline results
Train loss: 2.0531278200149536 {'accuracy': 0.463, 'precision_macro': 0.40900232402045056, 'recall_macro': 0.3195713655497666, 'f1_macro': 0.3099277526824259}
Val loss: 1.3288500208854674 {'accuracy': 0.6265, 'precision_macro': 0.45887587296325866, 'recall_macro': 0.4596022235952979, 'f1_macro': 0.4297621247576931}


In [27]:
import torch
import torch.nn as nn
from torchvision import models
from transformers import AutoModel
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

NUM_CLASSES = len(label2id)

class MultiModalConcat(nn.Module):
    def __init__(self, num_classes, text_model="distilbert-base-uncased", dropout=0.2):
        super().__init__()

        # Image encoder (ResNet18)
        img = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        img_dim = img.fc.in_features
        img.fc = nn.Identity()
        self.img_encoder = img
        self.img_proj = nn.Sequential(
            nn.Linear(img_dim, 256),
            nn.ReLU()
        )

        # Text encoder (DistilBERT)
        self.txt_encoder = AutoModel.from_pretrained(text_model)
        txt_dim = self.txt_encoder.config.hidden_size
        self.txt_proj = nn.Sequential(
            nn.Linear(txt_dim, 256),
            nn.ReLU()
        )

        # Fusion head
        self.head = nn.Sequential(
            nn.Linear(256 + 256, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )

    def forward(self, image, input_ids, attention_mask):
        v = self.img_proj(self.img_encoder(image))  # [B,256]
        t_out = self.txt_encoder(input_ids=input_ids, attention_mask=attention_mask)
        t = self.txt_proj(t_out.last_hidden_state[:, 0, :])  # CLS [B,256]
        h = torch.cat([v, t], dim=1)
        return self.head(h)

def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)
    return {"accuracy": acc, "precision_macro": p, "recall_macro": r, "f1_macro": f1}

def train_one_epoch_mm(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    all_true, all_pred = [], []

    for batch in tqdm(loader, desc="train (multimodal)"):
        image = batch["image"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label = batch["label"].to(device)

        optimizer.zero_grad()
        logits = model(image, input_ids, attention_mask)
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1).detach().cpu().tolist()
        all_pred.extend(preds)
        all_true.extend(label.detach().cpu().tolist())

    return total_loss / len(loader), compute_metrics(all_true, all_pred)

@torch.no_grad()
def eval_one_epoch_mm(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_true, all_pred = [], []

    for batch in tqdm(loader, desc="val (multimodal)"):
        image = batch["image"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label = batch["label"].to(device)

        logits = model(image, input_ids, attention_mask)
        loss = criterion(logits, label)

        total_loss += loss.item()
        preds = logits.argmax(dim=1).detach().cpu().tolist()
        all_pred.extend(preds)
        all_true.extend(label.detach().cpu().tolist())

    return total_loss / len(loader), compute_metrics(all_true, all_pred)

mm_model = MultiModalConcat(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(mm_model.parameters(), lr=2e-5, weight_decay=1e-2)

train_loss, train_metrics = train_one_epoch_mm(mm_model, train_loader, optimizer, criterion)
val_loss, val_metrics     = eval_one_epoch_mm(mm_model, val_loader, criterion)

print("\nMultimodal (concat) results")
print("Train loss:", train_loss, train_metrics)
print("Val loss:", val_loss, val_metrics)


Device: cuda


val (multimodal): 100%|██████████| 125/125 [00:14<00:00,  8.67it/s]


Multimodal (concat) results
Train loss: 2.204787401390076 {'accuracy': 0.4203, 'precision_macro': 0.3706692089215039, 'recall_macro': 0.25440751078127866, 'f1_macro': 0.2420619723727544}
Val loss: 1.3916817226409912 {'accuracy': 0.631, 'precision_macro': 0.470684929673428, 'recall_macro': 0.4490725077191158, 'f1_macro': 0.4254641320567494}


In [28]:
import os, json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support

model = mm_model
model.eval()

y_true, y_pred = [], []

with torch.no_grad():
    for batch in val_loader:
        image = batch["image"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label = batch["label"].to(device)

        logits = model(image, input_ids, attention_mask)
        preds = logits.argmax(dim=1)

        y_true.extend(label.cpu().numpy().tolist())
        y_pred.extend(preds.cpu().numpy().tolist())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Metrics
acc = accuracy_score(y_true, y_pred)
p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)

metrics = {
    "val_accuracy": float(acc),
    "val_precision_macro": float(p),
    "val_recall_macro": float(r),
    "val_f1_macro": float(f1),
    "model": "multimodal_concat"
}
print("Metrics:", metrics)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
os.makedirs("/content/results", exist_ok=True)

plt.figure(figsize=(10, 8))
plt.imshow(cm)
plt.title("Confusion Matrix (Multimodal Concat)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()
plt.tight_layout()
plt.savefig("/content/results/confusion_matrix_multimodal_concat.png", dpi=200)
plt.close()

print("Saved:", "/content/results/confusion_matrix_multimodal_concat.png")


report = classification_report(y_true, y_pred, digits=4, zero_division=0)
print("\nClassification Report (Val):\n")
print(report)

with open("/content/results/metrics_multimodal_concat.json", "w") as f:
    json.dump(metrics, f, indent=2)

with open("/content/results/classification_report_multimodal_concat.txt", "w") as f:
    f.write(report)

print("Saved:", "/content/results/metrics_multimodal_concat.json")
print("Saved:", "/content/results/classification_report_multimodal_concat.txt")


Metrics: {'val_accuracy': 0.631, 'val_precision_macro': 0.470684929673428, 'val_recall_macro': 0.4490725077191158, 'val_f1_macro': 0.4254641320567494, 'model': 'multimodal_concat'}
Saved: /content/results/confusion_matrix_multimodal_concat.png

Classification Report (Val):

              precision    recall  f1-score   support

           0     0.7500    0.4286    0.5455        77
           1     0.3971    0.4219    0.4091        64
           2     0.5833    0.2258    0.3256        31
           3     0.0000    0.0000    0.0000        20
           4     0.6410    0.7463    0.6897        67
           5     0.8229    0.9080    0.8634        87
           6     0.0000    0.0000    0.0000        19
           7     0.3254    0.6126    0.4250       111
           8     0.6667    0.0370    0.0702        54
           9     0.8194    0.9695    0.8881       131
          10     0.0000    0.0000    0.0000        29
          11     0.0000    0.0000    0.0000        57
          12     0.857

In [29]:
import pandas as pd
from IPython.display import display

results = [
    {
        "Model": "Image-only (ResNet18)",
        "Modality": "Image",
        "Val Accuracy": 0.468,
        "Val Macro-F1": 0.340
    },
    {
        "Model": "Text-only (DistilBERT)",
        "Modality": "Text",
        "Val Accuracy": 0.8305,
        "Val Macro-F1": 0.8070
    },
    {
        "Model": "Multimodal (Concat)",
        "Modality": "Image + Text",
        "Val Accuracy": 0.8490,
        "Val Macro-F1": 0.8211
    }
]

df_results = pd.DataFrame(results)

display(df_results.style
        .format({"Val Accuracy": "{:.4f}", "Val Macro-F1": "{:.4f}"})
        .set_caption("Table X: Comparison of Unimodal and Multimodal Models"))


,Model,Modality,Val Accuracy,Val Macro-F1
0,Image-only (ResNet18),Image,0.4680,0.3400
1,Text-only (DistilBERT),Text,0.8305,0.8070
2,Multimodal (Concat),Image + Text,0.8490,0.8211
